In [ ]:
merchants = ["Adidas","Airbnb","Aliexpress","Amazon","Apple Store","ASOS","Bath & Body Works","Booking.com","Boots","Burger King","Carrefour","Chipotle","Costa Coffee","Decathlon","Dia","Domino's Pizza",
              "Dunkin' Donuts","El Corte Inglés","Etsy","Expedia","Fnac","Groupon","H&M","Ikea","KFC","La Caixa","Lidl","Lonely Planet","Mango","McDonald's","MediaMarkt","Mercadona","Netflix",
              "Nike","Panera Bread","Papa John's","Patagonia","Pizza Hut","Red Lobster","Repsol","Samsung Store","Santander","Sephora","Skyscanner","Spotify","Starbucks","Subway","The North Face",
              "Thomas Cook","TGI Fridays","TripAdvisor","TUI","Uber","Vinted","Wish","Zalando","Zara"]
tags = [ "moda","e-commerce","viajes","belleza","electrónica","farmacia","restaurante","alojamiento","deportes","servicios","supermercado","banca","cafetería","transporte","entretenimiento","tienda","combustible"]
cities = ["Madrid","Sevilla","Barcelona","Valencia"]

In [ ]:
import csv

# Ruta del archivo CSV (ajusta según tu configuración)
archivo_csv = "transactions.csv"

# Nombre de la columna que deseas extraer
merchant = "Merchant"
tags = "Tag1;Tag2;Tag3"
cities = "City"

# Lista para almacenar los datos de la columna
merchants = []
tags = []
cities = "City"
try:
    # Abrir y leer el archivo CSV
    with open(archivo_csv, mode='r', encoding='utf-8') as archivo:
        lector = csv.DictReader(archivo)  # Lee el archivo como un diccionario
        for fila in lector:
            # Agregar el valor de la columna deseada a la lista
            merchants.append(fila[merchant])

    # Mostrar los datos obtenidos
    print(f"Datos de la columna '{columna_deseada}':", lista_datos)

except FileNotFoundError:
    print(f"El archivo '{archivo_csv}' no fue encontrado.")
except KeyError:
    print(f"La columna '{columna_deseada}' no existe en el archivo CSV.")
except Exception as e:
    print(f"Ocurrió un error: {e}")


In [ ]:
import base64
import re
import vertexai
from vertexai.generative_models import GenerativeModel, SafetySetting
from google.cloud import bigquery


def handle_input(input_string):
  # Check for date and/or amount
  month_match = re.search(r"\b(january|february|march|april|may|june|july|august|september|october|november|december|enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre|año|year)\b", input_string.lower())
  number_match = re.search(r'\d+', input_string)

  # Check for Tag and City mentions in the input
  tag_keywords = ["tag1", "tag2", "tag3"]  # List of possible tag columns
  city_keywords = ["city"]  # List for city
  merchant_keywords = ["merchant"] #List of merchant

  # Check if any tag or city keywords are in the input
  tags_detected = any(tag in input_string.lower() for tag in tag_keywords)
  city_detected = any(city in input_string.lower() for city in city_keywords)
  merchant_detected = any(merchant in input_string.lower() for merchant in merchant_keywords)

  # Check for multiple columns (Tag + City)
  columns = re.findall(r'(\w+): (\w+)', input_string)

  # If we detect both tag and city information, classify as NL2Q
  if (tags_detected and merchant_detected and city_detected) or (tags_detected and city_detected) or (tags_detected and merchant_detected) or (city_detected and merchant_detected) or number_match or month_match:
    return "NL2Q"  # Matches multiple columns (e.g., Tag and City) and date/amount
  else:
    return "StringMatching"  # Matches only one column or no clear pattern

def filter_input(input_string, merchants, tags, cities):
    # Split the input into words
    words = input_string.split()

    # Filter words based on known categories (merchants, tags, cities)
    filtered_words = [
        word for word in words
        if word in merchants or word in tags or word in cities
    ]

    return filtered_words

# Configuración para generar consultas SQL
def generate(user_input,process):
    vertexai.init(project="go-agl-poc-itedt-p01-poc", location="europe-west4")
    model = GenerativeModel(
        "gemini-1.5-flash-002",
        system_instruction=[process]
    )

    # Generar la consulta basada en la entrada del usuario
    responses = model.generate_content(
        [user_input],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    # Capturar el texto de la respuesta generada
    query_text = ""
    for response in responses:
        query_text += response.text

    return query_text

# Instrucciones del modelo para generar la consulta
#nl2q
process = """Create a query for our transaction table (go-agl-poc-itedt-p01-poc.ds_ab_poc.transactions), where we have the columns: Date, Amount, Merchant, Tag1, Tag2, Tag3, and City.
Depending on what the user asks, create a query with the corresponding fields. If the user enters a date convert it to type yyyy/mm/dd (eg: october filter: WHERE Date '2023-10-01' AND '2023-10-31' range in query),
and the different tag fields check in all of the columns if the filter is introduced. Do not add additional information, only the query. If you don't know if the field inputed is tag or merchant add all the conditions to the query with OR"""

# Configuración de generación
generation_config = {
    "max_output_tokens": 4045,
    "temperature": 1,
    "top_p": 0.95,
}

# Configuración de seguridad
safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
]
# Function to continuously process user input
def interactive_query_processing():
    while True:
        # Ask for user input
        user_input = input("Escribe tu pregunta: ")

        # Exit condition
        if user_input.lower() == 'exit':
            print("Exiting the program.")
            break

        # Process the input
        processing_method = handle_input(user_input)
        print(f"Processing method: {processing_method}")

        filtered_words = filter_input(user_input,merchants, tags, cities)
        print(f"Filtered words: {filtered_words}")

        # Generate the SQL query
        if processing_method == "StringMatching" and filtered_words:
            #query = f"SELECT * FROM `go-agl-poc-itedt-p01-poc.ds_ab_poc.transactions` WHERE Merchant LIKE '{user_input}' OR Tag1 LIKE '{user_input}' OR Tag2 LIKE '{user_input}' OR Tag3 LIKE '{user_input}' OR City LIKE '{user_input}'"
            # Split the input into words


            # Create a WHERE clause that checks for matches for each word across the specified columns
            conditions = []
            for word in filtered_words:
                condition = f"""
                Merchant LIKE '%{word}%' OR
                Tag1 LIKE '%{word}%' OR
                Tag2 LIKE '%{word}%' OR
                Tag3 LIKE '%{word}%' OR
                City LIKE '%{word}%'
                """
                conditions.append(f"({condition})")
            print("The conditions are", conditions)
            # Combine conditions with AND to ensure all words are matched
            where_clause = " AND ".join(conditions)

            # Generate the final query
            query = f"SELECT * FROM `go-agl-poc-itedt-p01-poc.ds_ab_poc.transactions` WHERE {where_clause}"

        elif processing_method == "StringMatching":
            query = generate(user_input, process)
            query = query.replace("```sql", "", 1)
            query = query.replace("```", "", 1)

        else:
          print("Sorry enter again:")
          interactive_query_processing()

        print("Consulta SQL generada:", query)

        # Execute the query on BigQuery
        client = bigquery.Client()

        # Execute the generated query
        query_job = client.query(query)

        results = query_job.result()

        # Obtener los nombres de las columnas de la consulta
        columns = results.schema
        column_names = [field.name for field in columns]

        # Imprimir los encabezados de la tabla
        # Alineamos las columnas a la izquierda y con un ancho de columna ajustado para que se vea bien
        header = " | ".join([f"{col:<20}" for col in column_names])
        print(header)
        print("-" * len(header))  # Una línea de separación

        # Imprimir los resultados en forma de tabla
        for row in results:
            row_values = [str(value) for value in row.values()]
            print(" | ".join([f"{value:<20}" for value in row_values]))

# Run the interactive process
interactive_query_processing()
